In [25]:
#import dependencies
import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scipy.stats as st
import os

In [26]:
# Read single year crime files and store into final data frame
dunkin_csv = "Resources/final_dunkin_loc.csv"
mcdonalds_csv = "Resources/final_mcd_loc.csv"
panera_csv = "Resources/final_panera_loc.csv"
starbucks_csv = "Resources/sbux_loc.csv"

# pd.read_csv('file.csv', index_col=0)
dunkin_df = pd.read_csv(dunkin_csv, index_col=0)
mcdonalds_df = pd.read_csv(mcdonalds_csv, index_col=0)
panera_df = pd.read_csv(panera_csv, index_col=0)
starbucks_df = pd.read_csv(starbucks_csv, index_col=0)

In [27]:
#Combine all csv files
frames = [dunkin_df, mcdonalds_df, panera_df, starbucks_df]
combine_df = pd.concat(frames)
combine_df

,strLocation,latitude,longitude,city,state,zip,Store
id,,,,,,,
2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
33 E Lake St IL 60101,1,41.931216,-87.987127,Addison,IL,60101,Dunkin'
...,...,...,...,...,...,...,...
49008-266131,157 & Center Grove Rd,38.787750,-89.978912,Edwardsville,IL,62025,Starbucks
13852-108653,Troy & Governor's Pkwy,38.788790,-89.951888,Edwardsville,IL,62025,Starbucks
76860-116533,Target Belleville T-2330,38.511200,-90.014600,Belleville,IL,62226,Starbucks


In [28]:
combine_df.drop_duplicates(subset=None, keep='first', inplace=True)
combine_df

,strLocation,latitude,longitude,city,state,zip,Store
id,,,,,,,
2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
33 E Lake St IL 60101,1,41.931216,-87.987127,Addison,IL,60101,Dunkin'
1750 W Lake St IL 60101,1,41.950451,-88.030146,Addison,IL,60101,Dunkin'
1319 East Algonquin Road IL 60102,1,42.161334,-88.276569,Algonquin,IL,60102,Dunkin'
2543 W Algonquin Rd IL 60102,1,42.175207,-88.338530,Algonquin,IL,60102,Dunkin'
...,...,...,...,...,...,...,...
49008-266131,157 & Center Grove Rd,38.787750,-89.978912,Edwardsville,IL,62025,Starbucks
13852-108653,Troy & Governor's Pkwy,38.788790,-89.951888,Edwardsville,IL,62025,Starbucks
76860-116533,Target Belleville T-2330,38.511200,-90.014600,Belleville,IL,62226,Starbucks


In [29]:
combine_df = combine_df.loc[combine_df['state']=='MO']
combine_df

,strLocation,latitude,longitude,city,state,zip,Store
id,,,,,,,
2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
3996 Jeffco Blvd MO 63010,1,38.405690,-90.376802,Arnold,MO,63010,McDonald's
130 Arnold Crossroads Ctr MO 63010,0.88,38.445494,-90.374454,Arnold,MO,63010,McDonald's
2201 Michigan Ave MO 63010,1,38.437040,-90.377478,Arnold,MO,63010,McDonald's
2005 Richardson Rd MO 63010,1,38.418379,-90.390784,Arnold,MO,63010,McDonald's
...,...,...,...,...,...,...,...
62568-295489,S Jefferson and Walnut,38.629806,-90.216753,St. Louis,MO,63103,Starbucks
75586-68060,Marriott St. Louis Grand,38.630655,-90.191852,St. Louis,MO,63101,Starbucks
63505-296800,Ballpark Village,38.624702,-90.193569,St. Louis,MO,63102,Starbucks


In [30]:
combine_df.reset_index(inplace=True)
combine_df

,id,strLocation,latitude,longitude,city,state,zip,Store
0,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
1,3996 Jeffco Blvd MO 63010,1,38.405690,-90.376802,Arnold,MO,63010,McDonald's
2,130 Arnold Crossroads Ctr MO 63010,0.88,38.445494,-90.374454,Arnold,MO,63010,McDonald's
3,2201 Michigan Ave MO 63010,1,38.437040,-90.377478,Arnold,MO,63010,McDonald's
4,2005 Richardson Rd MO 63010,1,38.418379,-90.390784,Arnold,MO,63010,McDonald's
...,...,...,...,...,...,...,...,...
313,62568-295489,S Jefferson and Walnut,38.629806,-90.216753,St. Louis,MO,63103,Starbucks
314,75586-68060,Marriott St. Louis Grand,38.630655,-90.191852,St. Louis,MO,63101,Starbucks
315,63505-296800,Ballpark Village,38.624702,-90.193569,St. Louis,MO,63102,Starbucks
316,75520-103839,St. Louis Hilton at the Ballpark,38.625172,-90.190579,St. Louis,MO,63102,Starbucks


In [31]:
combine_df['index_col']=combine_df.index
combine_df

C:\Users\Janelle Goddard\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,strLocation,latitude,longitude,city,state,zip,Store,index_col
0,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin',0
1,3996 Jeffco Blvd MO 63010,1,38.405690,-90.376802,Arnold,MO,63010,McDonald's,1
2,130 Arnold Crossroads Ctr MO 63010,0.88,38.445494,-90.374454,Arnold,MO,63010,McDonald's,2
3,2201 Michigan Ave MO 63010,1,38.437040,-90.377478,Arnold,MO,63010,McDonald's,3
4,2005 Richardson Rd MO 63010,1,38.418379,-90.390784,Arnold,MO,63010,McDonald's,4
...,...,...,...,...,...,...,...,...,...
313,62568-295489,S Jefferson and Walnut,38.629806,-90.216753,St. Louis,MO,63103,Starbucks,313
314,75586-68060,Marriott St. Louis Grand,38.630655,-90.191852,St. Louis,MO,63101,Starbucks,314
315,63505-296800,Ballpark Village,38.624702,-90.193569,St. Louis,MO,63102,Starbucks,315
316,75520-103839,St. Louis Hilton at the Ballpark,38.625172,-90.190579,St. Louis,MO,63102,Starbucks,316


In [32]:
combine_df[["latitude", "index_col"]]=combine_df[["latitude", "index_col"]].astype(str)
combine_df["new_latitude"] =combine_df["latitude"]+combine_df["index_col"]
combine_df

C:\Users\Janelle Goddard\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\Janelle Goddard\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,strLocation,latitude,longitude,city,state,zip,Store,index_col,new_latitude
0,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin',0,39.0750010
1,3996 Jeffco Blvd MO 63010,1,38.40569,-90.376802,Arnold,MO,63010,McDonald's,1,38.405691
2,130 Arnold Crossroads Ctr MO 63010,0.88,38.445494,-90.374454,Arnold,MO,63010,McDonald's,2,38.4454942
3,2201 Michigan Ave MO 63010,1,38.43704,-90.377478,Arnold,MO,63010,McDonald's,3,38.437043
4,2005 Richardson Rd MO 63010,1,38.418379,-90.390784,Arnold,MO,63010,McDonald's,4,38.4183794
...,...,...,...,...,...,...,...,...,...,...
313,62568-295489,S Jefferson and Walnut,38.629806,-90.216753,St. Louis,MO,63103,Starbucks,313,38.629806313
314,75586-68060,Marriott St. Louis Grand,38.630655,-90.191852,St. Louis,MO,63101,Starbucks,314,38.630655314
315,63505-296800,Ballpark Village,38.624702,-90.193569,St. Louis,MO,63102,Starbucks,315,38.624702315
316,75520-103839,St. Louis Hilton at the Ballpark,38.625172,-90.190579,St. Louis,MO,63102,Starbucks,316,38.625172316


In [33]:
combine_df = combine_df[["new_latitude", "longitude", "Store"]]
combine_df = combine_df.rename(columns={"new_latitude":"latitude"})
combine_df = combine_df[["latitude", "longitude", "Store"]]



In [34]:
combine_df

,latitude,longitude,Store
0,39.0750010,-94.379703,Dunkin'
1,38.405691,-90.376802,McDonald's
2,38.4454942,-90.374454,McDonald's
3,38.437043,-90.377478,McDonald's
4,38.4183794,-90.390784,McDonald's
...,...,...,...
313,38.629806313,-90.216753,Starbucks
314,38.630655314,-90.191852,Starbucks
315,38.624702315,-90.193569,Starbucks
316,38.625172316,-90.190579,Starbucks


In [35]:
# Push the remade DataFrame to a new CSV file
combine_df.to_csv("Resources/coffee_clean.csv",
                  encoding="utf-8", index=False, header=True)